In [1]:
!pip install -q transformers sentencepiece accelerate gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

print("⏳ Loading NLLB Translator... (This might take a minute)")
# 1. Load Translation Model
checkpoint = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Create the translation pipeline
translator = pipeline("translation", model=model, tokenizer=tokenizer, max_length=400)

print("All Models Loaded Successfully!")

⏳ Loading NLLB Translator... (This might take a minute)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


⏳ Loading Language Detector...


model.bin:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

All Models Loaded Successfully!


In [10]:
# 1. Install and Import the correct library
!pip install -q langdetect
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0 # Ensures consistent results

# 2. Define Language Codes (REQUIRED for the logic to work)
LANG_CODES = {
    "English": "eng_Latn",
    "Urdu": "urd_Arab",
    "Hindi": "hin_Deva",
    "French": "fra_Latn",
    "German": "deu_Latn",
    "Spanish": "spa_Latn",
    "Chinese": "zho_Hans",
    "Arabic": "ara_Arab"
}

AUTO_MAP = {
    "en": "eng_Latn",
    "ur": "urd_Arab",
    "hi": "hin_Deva",
    "fr": "fra_Latn",
    "de": "deu_Latn",
    "es": "spa_Latn",
    "zh": "zho_Hans",
    "ar": "ara_Arab"
}

# 3. The Logic Function (Using langdetect)
def translate_text(text, source_selection, target_selection):
    if not text:
        return ""

    # Handle Target
    target_code = LANG_CODES[target_selection]

    # Handle Source
    if source_selection == "Auto Detect":
        try:
            # Predict using langdetect (returns 'en', 'ur', 'hi', etc.)
            detected_iso = detect(text)

            # Map ISO to NLLB Code
            source_code = AUTO_MAP.get(detected_iso, "eng_Latn")
            label_prefix = f"**[Detected: {detected_iso.upper()} -> {source_code}]**\n\n"
        except Exception as e:
            # Fallback if detection fails (e.g. empty text or numbers)
            source_code = "eng_Latn"
            label_prefix = f"**[Detection Failed, assuming English]**\n\n"
    else:
        source_code = LANG_CODES[source_selection]
        label_prefix = ""

    # Run Translation
    translator.model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(target_code)
    output = translator(text, src_lang=source_code, tgt_lang=target_code)
    return label_prefix + output[0]['translation_text']

print("Final Logic Ready!")

Final Logic Ready!


In [11]:
import gradio as gr

# Create the Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌍 AI Universal Translator")
    gr.Markdown("Translate between **English, Urdu, Hindi, French, German, Spanish, Chinese, and Arabic**.")

    with gr.Row():
        # Input Settings
        source_lang = gr.Dropdown(
            choices=["Auto Detect"] + list(LANG_CODES.keys()),
            value="Auto Detect",
            label="Source Language"
        )
        target_lang = gr.Dropdown(
            choices=list(LANG_CODES.keys()),
            value="English",
            label="Target Language"
        )

    # Input/Output Areas
    input_text = gr.Textbox(placeholder="Type your text here...", label="Input Text", lines=3)
    output_text = gr.Textbox(label="Translated Text", lines=3, interactive=False)

    # Button
    translate_btn = gr.Button("Translate", variant="primary")

    # Action
    translate_btn.click(
        fn=translate_text,
        inputs=[input_text, source_lang, target_lang],
        outputs=output_text
    )

# Launch the App
demo.launch()

/tmp/ipython-input-2718749811.py:4: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9c8ed9b2dd7ce8931f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
